In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.neighbors import NearestNeighbors
import matplotlib
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import seaborn as sns
from matplotlib.colors import ListedColormap
from collections import defaultdict
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
import os
import shutil

from typing import Tuple, Any
import ipympl
%matplotlib ipympl
np.random.seed(23061998)
import random
import gzip
RANDOM_SEED = 23061998

plt.style.use('default')

In [ ]:
def extract_reps(confs_df: str, res_i_name: str, n_optimal: int):
    '''    Arguments:
    ---
        @ all_df: a pandas dataframe containing all conformations found in pdb for the given pair
        @ major_df: a pandas dataframe containing screened populated categories generated by the categorized_datapoints function

    Returns:
    ---
        @ a numpy array containing indices of the representative confs for each category. Indices point to the confs
        in all_df'''
    gaussian = True
    
    all_df = pd.read_csv(confs_df, index_col=0)
    samples = all_df.iloc[:,-4:].to_numpy() #training set is all data points in space

    scaler = MinMaxScaler()
    scaler.fit(samples)
    scaled_samples = scaler.transform(samples)

    if gaussian == True:
        optimal_gmm = GaussianMixture(n_components=n_optimal, random_state=RANDOM_SEED)
        optimal_gmm.fit(scaled_samples)
        labels = optimal_gmm.predict(scaled_samples)

    else:
        clustering = AgglomerativeClustering(n_clusters=n_optimal)
        labels = clustering.fit_predict(samples)
    # clustering.centroids

    all_df['clustering_lables'] = labels

    # clustering.centroids
    nearest_centroids = NearestCentroid()
    nearest_centroids.fit(scaled_samples, labels)
    centroids = nearest_centroids.centroids_
    #nearest_neightbor = NearestNeighbors(n_neighbors=1)
    # print(centroids.shape)
    inverse_centroid = scaler.inverse_transform(centroids)
    # print('X')
    #print(inverse_centroid)
    
    nearest_neightbor = NearestNeighbors(n_neighbors=1)
    rep_per_label = {} #parameters in 3 coord of representors
    rep_non_scaled = {} #indices in the all atom array
    rep_indices = []
    
    for label, centorid in enumerate(centroids):
        #print(centorid,label)
        labels_arr = all_df['clustering_lables'].to_numpy()
        idx = np.where(labels_arr == label)
        idx = np.ravel(idx)
                
        samples_for_label = scaled_samples[idx]
        non_scaled_samples = samples[idx]
        #now extract the index in indices list. that's nearest group's centoid:
        rep = nearest_neightbor.fit(samples_for_label).kneighbors(centorid.reshape(1,-1), return_distance=False)[:,0]
        #print(rep)
        rep_per_label[label] = samples_for_label[rep][0,:]
        
        rep_non_scaled[label] = non_scaled_samples[rep][0,:]
        
        int_rep = int(rep)
        curr_rep = idx[int_rep]
        rep_indices.append(curr_rep)


    reps_arr = []
    for key in range(len(rep_per_label)):
        reps_arr.append(rep_per_label[key])
    reps_arr = np.array(reps_arr)
    
    non_scaled_reps_arr = []
    for key in range(len(rep_non_scaled)):
        non_scaled_reps_arr.append(rep_non_scaled[key])
    non_scaled_reps_arr = np.array(non_scaled_reps_arr)
    #all members array:
    all_df.iloc[rep_indices,:4].to_csv(f'../csv_files/TRP_{res_i_name}_rep_confs.csv')
    
    if not os.path.isdir(f"../../TRP_{res_i_name}_xyz"):
        os.makedirs(f"../../TRP_{res_i_name}_xyz")
    all_confs_df = pd.read_csv(f'../csv_files/TRP_{res_i_name}_rep_confs.csv', index_col=0)
    for pdb_file in all_confs_df.iloc[:,0].values.tolist():
        gzip_file = f"../../{pdb_file}.gz"
        with gzip.open(gzip_file, 'rb') as f_in:
            with open(f"../../TRP_{res_i_name}_xyz/{pdb_file}", 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    return centroids, reps_arr, non_scaled_reps_arr, samples, labels             

def find_representative_confs(data_file: str, n_optimal: int, res_i_name) -> np.ndarray:
    """Finds a single closest geometry point in the space of P, D (rescaled), Ttheta1, Ttheta2.
    The representative points should be closest to the cluster representing the (P,D,T1,T2) spatial occupancies.

    Params:
        data_file (str) - name/path of a csv datafile containing all conformations found in pdb for the given pair and their 4D params.
        n_optimal (int) - number of clusters to assume.
        res_i_name (str) - The first residue here is TRP, so res_i_name represents the name of the pair residue of TRP.

    Returns:
        np.ndarray: A numpy array containing indices of the representative confs for each category. Indices point to the confs
        in all_df.
    """
    # data_file = 'fixed_TRP_cationpi.csv'
    centroids, reps_arr, non_scaled_reps_arr, samples, labels = extract_reps(data_file, res_i_name, n_optimal)
    
    return centroids, reps_arr, non_scaled_reps_arr, samples, labels



In [5]:
n_optimal = 154
res_j_name = 'TYR'
data_file = f'../csv_files/TRP_{res_j_name}.csv'
cents,reps_scaled,reps_origin,samples,labels = find_representative_confs(data_file, n_optimal, res_j_name)